# NMNIST NetX Tutorial
Hello everyone, in this tutorial of lava-dl, we are going to convert the model trained in the Slayer NMNIST tutorial to lava processes and run it on Loihi simulator.

Network excange module is available as `lava.lib.dl.netx.{hdf5, blocks, utils}`.
* `hdf5` implements automatic network generation.
* `blocks` implements individual layer blocks.
* `utils` implements hdf5 reading utilities. 

## Import Phase
First, let us import the libraries needed for this tutorial. If you want to investigate why they are for, you can right-click the module and go to the declaration.

In [1]:

import numpy as np
import random

#Import io module
from lava.proc import io

# Import Process level primitives
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort
# Import parent classes for ProcessModels
from lava.magma.core.model.py.model import PyLoihiProcessModel
# Import ProcessModel ports, data-types
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
# Import execution protocol and hardware resources
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol

from lava.magma.core.resources import GPU #You can change it to the CPU if you wish.
# Import decorators
from lava.magma.core.decorator import implements, requires
# Import Compile and run modules
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

#Import network exchange module  
from lava.lib.dl import netx
# Import Dataset  
from nmnist import NMNISTDataset

## Network Initiation
Now, we will import the trained hdf5 file from the 'Trained' folder and print it before and after the exchange happens. In the tutorial folder, it is already included. If you wish, you can replace that one with the one you have trained with the Slayer NMNIST tutorial.

In [2]:
net = netx.hdf5.Network(net_config='Trained/network.net')
print(net)

print(f'There are {len(net)} layers in network:')
for l in net.layers:
    print(f'{l.block:5s}: {l.name:10s}, shape : {l.shape}')


|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Dense     |    1|    1|  512|     |     |     |     |     |True |
|Dense     |    1|    1|  512|     |     |     |     |     |True |
|Dense     |    1|    1|   10|     |     |     |     |     |False|
There are 3 layers in network:
Dense: Process_1 , shape : (512,)
Dense: Process_4 , shape : (512,)
Dense: Process_7 , shape : (10,)


## Dataset Instance Creation:
Here we initiate our dataset instance. But this NMNIST code is a modified version of the original ( [Reference](https://www.frontiersin.org/articles/10.3389/fnins.2015.00437/full)  ) NMNIST dataset code. However, credentials are conserved. If you experience a download issue, you can copy and paste the dataset available in the Slayer NMNIST training tutorial.

In [3]:
data_set = NMNISTDataset(
    path='data', 
    train=False, # This mark will give us the test set of the Dataset
)

Now we will specify some of our variables and choose random samples from the dataset instance.

In [4]:
num_images = 25 # You can change this value for your run.

num_steps_per_image = 300 #This value is adjusted for the NMNIST dataset. 300ms of events will be supplied.

num_steps = num_images * num_steps_per_image

idx = np.random.choice(np.arange(len(data_set.samples)),num_images,0)

## Process Definitions
Our system will consist of 3 main parts. 
### First Part: Spike Input
The first part will be the part in which Spike Inputs are supplied to the network.
### Second Part: Classifier Network
We have already created this part by converting our model by using lava.dl.NetX
### Third Part: Output
In this last part, our output process will accumulate the outputs of the network and give us which prediction is made. Also, ground truth data also will be supplied to this process by Spike Input.

In [5]:

class SpikeInput(AbstractProcess):
    """Reads image data from the NMNIST dataset """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        n_img = kwargs.pop('num_images', 25)
        n_steps_img = kwargs.pop('num_steps_per_image', 300)
        shape = (2312,)
        self.spikes_out = OutPort(shape=shape)  # Input spikes to the classifier
        self.label_out = OutPort(shape=(1,))  # Ground truth labels to OutputProc
        self.num_images = Var(shape=(1,), init=n_img)
        self.num_steps_per_image = Var(shape=(1,), init=n_steps_img)
        self.input_img = Var(shape=shape)
        self.ground_truth_label = Var(shape=(1,))
        self.v = Var(shape=shape, init=0)
        self.vth = Var(shape=(1,), init=kwargs['vth'])

class OutputProcess(AbstractProcess):
    """Process to gather spikes from 10 output LIF neurons and interpret the
    highest spiking rate as the classifier output"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = (10,)
        n_img = kwargs.pop('num_images', 25)
        n_steps_img  = kwargs.pop('num_steps_per_image', 300)
        self.num_images = Var(shape=(1,), init=n_img)
        self.spikes_in = InPort(shape=shape)
        self.label_in = InPort(shape=(1,))
        self.spikes_accum = Var(shape=shape)  # Accumulated spikes for classification
        self.num_steps_per_image = Var(shape=(1,), init=n_steps_img )
        self.pred_labels = Var(shape=(n_img,))
        self.gt_labels = Var(shape=(n_img,))

## Process Model Definitions
Now we will define Process models for the processes we have defined previously.  

### Input Process Model
This model will run in the Input process to supply the network with the data. The important part is that this model will supply the next event instance of the same number in every run step and will change to a different number every 300 steps. Decorators are used to specifying which protocol the process should use and which resource should be utilized.

In [6]:
@implements(proc=SpikeInput, protocol=LoihiProtocol)
@requires(GPU) #You can change it to the CPU if you wish.
class PySpikeInputModel(PyLoihiProcessModel):
    num_images: int = LavaPyType(int, int, precision=32)
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    label_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, np.int32,
                                      precision=32)
    num_steps_per_image: int = LavaPyType(int, int, precision=32)
    input_img: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    ground_truth_label: int = LavaPyType(int, int, precision=32)
    v: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    vth: int = LavaPyType(int, int, precision=32)
    
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.nmnist_dataset = data_set
        self.curr_img_id = 0
  
        self.idx = idx
    def post_guard(self):
        
        if self.time_step % self.num_steps_per_image == 0:
            return True
            
        else:
            img ,self.ground_truth_label = self.nmnist_dataset[self.idx[self.curr_img_id]]


            self.input_img = img[:, self.time_step % self.num_steps_per_image].astype(np.int32)

        return False

    def run_post_mgmt(self):
        """Post-Management phase: executed only when guard function above 
        returns True.
        """
        img ,self.ground_truth_label = self.nmnist_dataset[self.idx[self.curr_img_id]]

        self.input_img = img[:,0].astype(np.int32) 
        self.v = np.zeros(self.v.shape)
        self.label_out.send(np.array([self.ground_truth_label]))
        self.curr_img_id += 1

    def run_spk(self):
        """Spiking phase: executed unconditionally at every time-step
        """
        self.v[:] = self.v + self.input_img
        s_out = self.v > self.vth
        self.v[s_out] = 0  # reset voltage to 0 after a spike
        self.spikes_out.send(s_out)

### Output Process Model
This model will run in the Output Process. Here this model makes the process to detect the most fired neuron of the output layer, so the prediction is obtained. Again, decorators are used to specifying which protocol the process should use and which resource should be utilized.

In [7]:


@implements(proc=OutputProcess, protocol=LoihiProtocol)
@requires(GPU) #You can change it to the CPU if you wish.
class PyOutputProcessModel(PyLoihiProcessModel):
    label_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int, precision=32)
    spikes_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    num_images: int = LavaPyType(int, int, precision=32)
    spikes_accum: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=32)
    num_steps_per_image: int = LavaPyType(int, int, precision=32)
    pred_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    gt_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
        
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.current_img_id = 0

    def post_guard(self):
        """Guard function for PostManagement phase.
        """
        if self.time_step % self.num_steps_per_image == 0 and \
                self.time_step > 1:
            return True
        return False

    def run_post_mgmt(self):
        """Post-Management phase: executed only when guard function above 
        returns True.
        """
        gt_label = self.label_in.recv()
        pred_label = np.argmax(self.spikes_accum)
        self.gt_labels[self.current_img_id] = gt_label
        self.pred_labels[self.current_img_id] = pred_label
        self.current_img_id += 1
        self.spikes_accum = np.zeros_like(self.spikes_accum)

    def run_spk(self):
        """Spiking phase: executed unconditionally at every time-step
        """
        spk_in = self.spikes_in.recv()
        self.spikes_accum = self.spikes_accum + spk_in

### Process Creation
Here we are creating the objects of the processes we have just defined.

In [8]:
spike_input = SpikeInput(num_images=num_images,
                         num_steps_per_image=num_steps_per_image,
                         vth=1)
output_proc = OutputProcess(num_images=num_images)

## Connection of Processes
As we have defined and created our processes, we will connect them to each other to be functionalized.

In [9]:
# Connect Processes
spike_input.spikes_out.connect(net.in_layer.synapse.s_in) # Spike Input layer to Network
spike_input.label_out.connect(output_proc.label_in) # Spike Input layer to Output layer for the ground truth
net.out_layer.out.connect(output_proc.spikes_in) # Network output to the output layer

## Run Phase
Now we have come to the sweetest part of our tutorial, running and showing the results. Here in the for loop number of samples are processed with the steps specified. Then the results are printed.

In [10]:
for img_id in range(num_images):
    print(f"\rCurrent image: {img_id+1}", end="")
    # Run each event-inference for fixed number of steps
    net.run(
        condition=RunSteps(num_steps=num_steps_per_image,),
        run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                             select_tag='fixed_pt'))
    

ground_truth = output_proc.gt_labels.get().astype(np.int32)
predictions = output_proc.pred_labels.get().astype(np.int32)

net.stop()
accuracy = np.sum(ground_truth==predictions)/ground_truth.size * 100

print(f"\nGround truth: {ground_truth}\n"
      f"Predictions : {predictions}\n"
      f"Accuracy    : {accuracy}")

Current image: 25
Ground truth: [3 5 1 0 5 4 8 4 4 6 7 6 4 3 7 2 6 1 9 7 5 6 6 8 4]
Predictions : [3 5 1 0 5 4 8 4 4 6 7 6 4 3 7 2 6 1 9 7 5 6 6 8 4]
Accuracy    : 100.0


# Conclusion

This was the end of our tutorial. Trusting your accuracy is high, I hope you have enjoyed it.
### Credit
This tutorial is created by Ahmet Akman, undergraduate student at [Middle East Technical University](metu.edu.tr), [Electrical ,and Electronics Engineering Department](eee.metu.edu.tr) , and undergraduate researcher at [METU Center for Image Analysis](http://ogam.metu.edu.tr/en/) For further information follow [github](github.com/ahmetakman) , [resume](dar.vin/ahmetakman_resume) and [linkedin](https://www.linkedin.com/in/ahmet-akman-039b05148/).